# Langchain Getting Started

This curated notebook demonstrates langchain getting started.

## Contents
1. Setup
2. Tutorial
3. Next Steps


# Getting Started with LangChain


In [ ]:
# TODO: describe cell
!pip install langchain
!pip install -qU langchain-groq
!pip install -qU langchain-openai
!pip install python_dotenv

## Getting started


In [ ]:
# TODO: describe cell
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
import os

In [ ]:
# TODO: describe cell
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# TODO: describe cell
gpt4o = ChatOpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
    model_name = "gpt-4o-mini")

In [ ]:
# TODO: describe cell
messages = [
    (
        "system",
        "Sei un chatbot assistente di cucina. Aiuta l'utente con le ricette.",
    ),
    ("human", "A che temperatura cuoce la pizza?"),
]

gpt_ping = gpt4o.invoke(messages)
print(gpt_ping)

## Prompting Template

In [ ]:
# TODO: describe cell
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Dimmi tre razze appartenti a alla specie {specie}")

prompt_template.invoke({"specie": "squali"})

In [ ]:
# TODO: describe cell
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Sei un chatbot esperto di zoologia"),
    ("user", "Dimmi tre razze appartenti a alla specie {specie}")
])

prompt_template.invoke({"specie": "dinosauri"})

## LangChain Expression Language (LCEL)


Double prompt to improve response

In [ ]:
# TODO: describe cell
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai la piscina?
User: Era molto piccola e troppo calda.
"""

standard_response = """ Grazie mille per il tuo feedback. Faremo il nostro meglio per migliorarci in futuro.
"""

PROMPT_INTERACTION_1 = """ Sei un chatbot assistente in un servizio clienti di un hotel. Stai avendo una conversazione con un cliente:
```{chat_history}```.
Genera una risposta appropriata per continuare la conversazione, adattando questa risposta standard:
Risposta standard: ```{standard_response}```.
"""

PROMPT_REFINE_1 = """ Sei un chatbot assistente in un servizio clienti di un hotel. Stai avendo una conversazione con un cliente.
La tua ultima risposta è stata questa ```{first_response}```.
Elimina eventuali domande e assicurati che sia simile a questa risposta standard: ```{standard_response}```.
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response"], template=PROMPT_INTERACTION_1)
prompt_clean_questions = PromptTemplate(input_variables=["first_response","standard_response"], template=PROMPT_REFINE_1)
std_resp = PromptTemplate.from_template(standard_response)

chain_one = prompt_start | gpt4o | StrOutputParser()
chain_two = prompt_clean_questions | gpt4o | StrOutputParser()

final_chain =({"first_response" : chain_one, "standard_response": std_resp}
        | RunnablePassthrough.assign(final_response = chain_two))

result = final_chain.invoke({"chat_history": chat_history, "standard_response": standard_response})

print(result['final_response'])

Customizing focus of response

In [ ]:
# TODO: describe cell
chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai?
User: Era molto piccola e troppo calda.
Chatbot: In che modo pensi potremmo migliorarci la prossima volta?
User: Un servizio di prenotazioni aiuterebbe magari
"""

standard_response = """ Grazie per il tuo feedback, ottimo consiglio.
Faremo il possibile per migliorare il servizio piscina, ci spiace non sia stato all'altezza delle tue aspettative! """

user_first_impression = """Era molto piccola e troppo calda. """
user_goal = """Un servizio di prenotazioni aiuterebbe magari"""

PROMPT_INTERACTION_F = """Sei un chatbot assistente che sta aiutando un cliente insoddisfatto riguardo a un servizio dell'hotel.
Stai avendo una conversazione: ```{chat_history}```.
Genera una risposta appropriata per continuare questa conversazione, adattando questa risposta standard.
Risposta standard: ```{standard_response}```
L'utente ha detto questo sull'esperienza: {user_first_impression}:
L'utente ti ha detto che spera di vedere miglioramenti: ```{user_goal}```
Scrivi solo la risposta.
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response", "user_first_impression", "user_goal"], template=PROMPT_INTERACTION_F)

chain_one = prompt_start | gpt4o | StrOutputParser()

result = chain_one.invoke({"chat_history": chat_history, "standard_response": standard_response,
                           "user_first_impression": user_first_impression, "user_goal": user_goal })

print(result)

In [ ]:
# TODO: describe cell

chat_history = """
Chatbot: Durante il tuo soggiorno, quale è il servizio che ti è piaciuto meno?
User: Piscina
Chatbot: Come mai?
User: Era molto piccola e troppo calda.
Chatbot: In che modo pensi potremmo migliorarci la prossima volta?
User: Un servizio di prenotazioni aiuterebbe magari
"""

user_first_impression = """Era molto piccola e troppo calda. """
user_response = """Un servizio di prenotazioni aiuterebbe magari"""


PROMPT_GOAL = """Sei un chatbot assistente che sta aiutando un cliente insoddisfatto riguardo a un servizio dell'hotel.
Stai avendo una conversazione: ```{chat_history}```.
Qual è lo scopo dell'utente? In che modo aiuterebbe a migliorare?
```{user_response}```
"""

prompt_start= PromptTemplate(input_variables=["chat_history","standard_response", "user_response"], template=PROMPT_GOAL)

chain_one = prompt_start | llama3 | StrOutputParser()

user_goal = chain_one.invoke({"chat_history": chat_history, "user_response": user_response })

print(user_goal)